In [1]:
# データ分析・操作用ライブラリ
import numpy as np
import pandas as pd

# NLP用ライブラリ
import MeCab,re
from sklearn.feature_extraction.text import TfidfVectorizer

# スコア評価用ライブラリ
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score


# エラー表示の抑制
import warnings

In [2]:
# 表示関連
# DataFrameの列数設定
pd.set_option('display.max_columns', 500)
pd.set_option("display.max_rows", 500)
warnings.filterwarnings('ignore')

In [3]:
# テキストデータの読み込み
wikiData = pd.read_csv("csv/train_cleaned.csv", encoding="utf-8")

# 読み込みデータの表示
wikiData.head()

,text,target
0,ドール・フード・カンパニー ドール・フード・カンパニー（Dole Food Compan...,1
1,南ヶ丘牧 南ヶ丘牧（みなみがおかぼくじょう）は、栃那須原に拠を置く酪農経営企。現在4つ...,1
2,岩瀬牧 岩瀬牧（いわせぼくじょう）は、福島岩瀬郡鏡石にある牧である。 鏡石シンボルソン...,1
3,"クラギ クラギ（くらぎ、""KURAGI Co.Ltd."" ）は、重を心に「農」を展開する...",1
4,"ハッピーネモファーム ハッピーネモファーム（""""）は、北海浦河郡浦河にある競走馬（サラブ...",1


In [4]:
# Numpy配列へ変換
X = wikiData.text.values
y = wikiData.target.values

In [5]:
# トレーニングデータ、テストデータの分割
X_train, X_test, y_train, y_test = train_test_split(X , y, test_size=0.2, random_state=42, stratify=y)

In [6]:
# 形態素解析用関数
def tokenize(text):
    tokens = []
    tagger = MeCab.Tagger( "-Ochasen" )
    node = tagger.parseToNode(text)
    while node:
        if node.feature.split(",")[0] == "名詞":
                replace_node = re.sub( re.compile( "[!-/:-@[-`{-~]" ), "", node.surface )
                if replace_node != "" and replace_node != " ":
                    tokens.append(replace_node)
        node = node.next
    return tokens

In [7]:
# 形態素解析を実行
vectorizer = TfidfVectorizer(tokenizer=tokenize)
vectorizer.fit(X)
train_matrix = vectorizer.transform(X_train)
test_matrix = vectorizer.transform(X_test)

In [8]:
# ナイーブベイズ
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()
clf.fit(train_matrix, y_train)
cv_scores = cross_val_score(clf, train_matrix, y_train, cv=5)

In [9]:
print("Training score：" + str(clf.score(train_matrix, y_train)))
print("Cross-Validation score：" + str(np.mean(cv_scores)))
print("Test score：" + str(clf.score(test_matrix, y_test)))

Training score：0.492146596859
Cross-Validation score：0.401495054042
Test score：0.39965095986


In [10]:
# ランダムフォレスト
from sklearn.ensemble import RandomForestClassifier

clf2 = RandomForestClassifier(n_estimators=50,random_state=42)
clf2.fit(train_matrix, y_train)
cv_scores = cross_val_score(clf2, train_matrix, y_train, cv=5)

In [11]:
print("Training score：" + str(clf2.score(train_matrix, y_train)))
print("Cross-Validation score：" + str(np.mean(cv_scores)))
print("Test score：" + str(clf2.score(test_matrix, y_test)))

Training score：1.0
Cross-Validation score：0.66221177878
Test score：0.657068062827


In [12]:
# Light GBM
import lightgbm as lgb

clf3 = lgb.LGBMClassifier(objective='multiclass', num_class=33)
clf3.fit(train_matrix, y_train)
cv_scores = cross_val_score(clf3, train_matrix, y_train, cv=5)

In [13]:
print("Training score：" + str(clf3.score(train_matrix, y_train)))
print("Cross-Validation score：" + str(np.mean(cv_scores)))
print("Test score：" + str(clf3.score(test_matrix, y_test)))

Training score：0.999127399651
Cross-Validation score：0.72687843
Test score：0.734729493892
